In [8]:
import numpy as np
import pandas as pd
import torch

for m in [np, pd, torch]:
    print(m.__name__, m.__version__)

from torch import nn, optim
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from funcs.factor_utils import FactorUtil

import warnings
warnings.filterwarnings('ignore')

numpy 1.26.4
pandas 2.2.3
torch 2.2.2


In [4]:
# 模拟参数
num_samples = 1000      # 样本数
seq_len = 20            # 时间序列长度
input_dim = 5           # 每个时间步的特征维度
num_classes = 3         # 分类数（A, B, C）

# 随机生成时间序列特征（形状: [num_samples, seq_len, input_dim]）
X = np.random.randn(num_samples, seq_len, input_dim).astype(np.float32)

# 随机生成标签（A=0, B=1, C=2）
y = np.random.choice(num_classes, size=num_samples)

print("特征 X 形状:", X.shape)   # (1000, 20, 5)
print("标签 y 形状:", y.shape)   # (1000,)

特征 X 形状: (1000, 20, 5)
标签 y 形状: (1000,)


In [6]:
# 位置编码（Positional Encoding）
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=500):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)
        
class TimeSeriesTransformerClassifier(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, num_classes, dropout=0.5):
        super().__init__()
        self.model_dim = model_dim

        # 输入映射层
        self.input_map = nn.Linear(input_dim, model_dim)

        # 位置编码
        self.pos_encoder = PositionalEncoding(model_dim, dropout)

        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, dim_feedforward=2 * model_dim)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # 分类头
        self.classifier = nn.Sequential(
            nn.Linear(model_dim * seq_len, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, num_classes)
        )

    def forward(self, src):
        # src shape: (batch_size, seq_len, input_dim)
        src = self.input_map(src)  # (batch_size, seq_len, model_dim)
        src = src.permute(1, 0, 2)  # (seq_len, batch_size, model_dim)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)  # (seq_len, batch_size, model_dim)
        output = output.permute(1, 0, 2)  # (batch_size, seq_len, model_dim)
        output = output.reshape(output.size(0), -1)  # (batch_size, seq_len * model_dim)
        logits = self.classifier(output)  # (batch_size, num_classes)
        return logits


In [9]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# 转为 Dataset 和 DataLoader
dataset = TimeSeriesDataset(X, y)
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20):
    model.to(device)
    best_acc = 0.0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        all_preds, all_labels = [], []

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        acc = accuracy_score(all_labels, all_preds)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}, Accuracy: {acc*100:.2f}%")

        # 验证
        model.eval()
        with torch.no_grad():
            val_preds, val_labels = [], []
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                preds = torch.argmax(outputs, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
            val_acc = accuracy_score(val_labels, val_preds)
            print(f"Validation Accuracy: {val_acc*100:.2f}%\n")

In [11]:
# 初始化模型
model = TimeSeriesTransformerClassifier(
    input_dim=5,
    model_dim=64,
    num_heads=4,
    num_layers=2,
    num_classes=3
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 开始训练
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)

Epoch 1/10, Loss: 28.9810, Accuracy: 33.88%
Validation Accuracy: 34.00%

Epoch 2/10, Loss: 27.9119, Accuracy: 35.00%
Validation Accuracy: 35.00%

Epoch 3/10, Loss: 27.5338, Accuracy: 38.00%
Validation Accuracy: 34.00%

Epoch 4/10, Loss: 27.3944, Accuracy: 37.38%
Validation Accuracy: 31.50%

Epoch 5/10, Loss: 27.1174, Accuracy: 40.25%
Validation Accuracy: 36.00%

Epoch 6/10, Loss: 27.0275, Accuracy: 40.75%
Validation Accuracy: 37.50%

Epoch 7/10, Loss: 26.5626, Accuracy: 41.25%
Validation Accuracy: 34.00%

Epoch 8/10, Loss: 25.7039, Accuracy: 47.75%
Validation Accuracy: 33.00%

Epoch 9/10, Loss: 25.1890, Accuracy: 47.88%
Validation Accuracy: 32.00%

Epoch 10/10, Loss: 25.0542, Accuracy: 52.50%
Validation Accuracy: 37.00%

